In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import applications
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
import cv2
import numpy as np

Using TensorFlow backend.
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

# Parse do dataset e separação entre conjuntos de teste e treinamento

In [2]:
# auxiliary function
def depth_two_uint8_to_float(top_bits, bottom_bits):
    """ Converts a RGB-coded depth into float valued depth. """
    depth_map = (top_bits * 2**8 + bottom_bits).astype('float32')
    depth_map /= float(2**16 - 1)
    depth_map *= 5.0
    return depth_map

set = 'evaluation'
# load annotations of this set
with open(os.path.join(set, 'anno_%s.pickle' % set), 'rb') as fi:
    anno_all = pickle.load(fi)
X_2d = []
X = []
y = []
NUM_SAMPLES = 5000

i = 0

items = anno_all.items()
img_height = img_width = 320
# iterate samples of the set
for sample_id, anno in items:
    # load data
    image = cv2.imread(os.path.join(set, 'color', '%.5d.png' % sample_id))
    #image = cv2.resize(image, None, fx = 0.5, fy = 0.5)
    image_reshaped = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_reshaped = np.reshape(image_reshaped, (img_height * img_width))
    
    mask = cv2.imread(os.path.join(set, 'mask', '%.5d.png' % sample_id), cv2.IMREAD_GRAYSCALE)
    mask = cv2.normalize(mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    #mask = cv2.resize(mask, None, fx = 0.5, fy = 0.5)
    mask = np.reshape(mask, (img_height * img_width))

    X_2d.append(image)
    X.append(image_reshaped)
    y.append(mask)

    i += 1
    
    if i >= NUM_SAMPLES:
        break


X = np.array(X).astype('float32')
X_2d = np.array(X_2d).astype('float32')
y = np.array(y).astype('float32')

print(X.shape)
print(X_2d.shape)
print(y.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'evaluation\\anno_evaluation.pickle'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# Instanciando o modelo Resnet50V2
base_model = applications.resnet_v2.ResNet50V2(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# Aplicando pooling e dropout no output
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)

batch_size = 16
num_classes = 4096
epochs = 20
data_augmentation = False
# Gerando camada de output e instanciando o modelo
predictions = Dense(num_classes, activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=predictions)

from keras.optimizers import SGD, Adam

opt = keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-5)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.layers import Input, Dense

input_shape = img_height * img_width

input_img = Input(shape=(input_shape,))
encoded = Dense(8192, activation='relu')(input_img)
encoded = Dense(1024, activation='relu')(encoded)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu')(encoded)

decoded = Dense(256, activation='relu')(decoded)
encoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1024, activation='relu')(decoded)
encoded = Dense(8192, activation='relu')(decoded)
decoded = Dense(input_shape, activation='sigmoid')(decoded)
autoencoder = Model(input_img, decoded)

opt = keras.optimizers.RMSprop(learning_rate=1e-6, decay=1e-6)
autoencoder.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
autoencoder.fit(X_train, y_train,
                epochs=250,
                batch_size=128,
                shuffle=True,
                validation_data=(X_validation, y_validation))

In [ ]:
acc = autoencoder.evaluate(X_test, y_test)
print("Test accuracy:" + str(acc[1]))
print("Loss:" + str(acc[0]))

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
ynew = autoencoder.predict(X_train)
ynew[0].shape


In [ ]:
y_train.shape


In [ ]:
for i in range(len(X_train)):
    image = image * 255
    image = np.reshape(ynew[i], (img_height, img_width))
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    cv2.imwrite("./results/image" + str(i) + ".jpg", image)
    
    train_image = np.reshape(y_train[i], (img_height, img_width))
    train_image = cv2.normalize(train_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    cv2.imshow("train", train_image)
    cv2.imshow("window",image)

    k = cv2.waitKey(0)
    print(k)
    if k==98:    # Esc key to stop
        break


In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_2d, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train.shape
y_train.shape

In [ ]:
num_classes = img_height * img_width
print(num_classes)
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
                epochs=100,
                batch_size=8,
                shuffle=True)

In [ ]:
acc = model.evaluate(x_test, y_test)
print("Test accuracy:" + str(acc[1]))
print("Loss:" + str(acc[0]))